In [160]:
import azure.functions as func
import pandas as pd
import numpy as np
import fuzzywuzzy.fuzz as fuzz
import os
import json


df=pd.read_csv("df-en-final.csv")
user_ingredients='egg,onion,tomato,rice,turmeric powder (haldi),red chilli powder,salt'
df["Ingredient-count"]=df["P-Ingredients"].apply(lambda x:len(x.split(",")))
df=df[df["Ingredient-count"]>4]
df.reset_index(drop=True,inplace=True)

def CookMan(df,user_ingredients):
    
    class Recipe: 
   
 
        def __init__(self, title,ing,time,missing,imgurl,cuisine,instructions): 
            self.title = title 
            self.ingredients=ing
            self.time=time
            self.missing=missing
            self.imgurl=imgurl 
            self.cuisine=cuisine
            self.instructions=instructions            
        def dump(self):
            return {"RecipeList": {'title': self.title,
                               'ingredients': self.ingredients,
                               'time': self.time,
                               'missing': self.missing,
                               'imgurl':self.imgurl,
                               'cuisine':self.cuisine,
                               'instructions':self.instructions }}


    def Cook2(Pingredients):    
        "Comparing the Recipe Ingredients and User Ingredients and decreasing count value for each Recipe ingredient missing.\
        Count describes the no. of ingredients missing from user ingredients to make that recipe."
        flag,count=0,0
        ulcount=0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if i==j.strip():               
                    flag=1
                    ulcount=1
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
            if count<-10:
                return count
        if(ulcount==0):
            return count-1
        else:
            return count

    def MainCook(Pingredients):    
        flag,count=0,0
        ullcount=0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j.strip())>80:                 
                    flag=1 
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
        if(ullcount==0):
            return count-1
        else:
            return count        
    
    
    df["flag"]=df["P-Ingredients"].apply(Cook2)  
    df=df.sort_values(by="flag",ascending=False).head(100)
    df["flag2"]=df["P-Ingredients"].apply(MainCook)
    df=df.sort_values(by="flag2",ascending=False)

    def Missing(Pingredients):
        flag=0
        miss=""
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j.strip())>80:                    
                    flag=1 
                    break
            if (flag==0):
                miss+=i+","
                
            else:
                flag=0
        miss = miss[:-1]
        return miss
    df["Missing"]=df["P-Ingredients"].apply(Missing)
    df.drop(columns=["flag2","flag","P-Ingredients","URL"],inplace=True)
    df.reset_index(inplace=True,drop=True)
    Cookdf=df.head(30)
    mylist=[]
    for i in range(5):
        title = Cookdf.loc[i,"TranslatedRecipeName"]
        ing=Cookdf.loc[i,"TranslatedIngredients"]
        time=int(Cookdf.loc[i,"TotalTimeInMins"])
        missing=Cookdf.loc[i,"Missing"]
        imgurl = Cookdf.loc[i,"image-url"]
        cuisine=Cookdf.loc[i,"Cuisine"]
        instructions=Cookdf.loc[i,"TranslatedInstructions"]
        mylist.append(Recipe(title,ing,time,missing,imgurl,cuisine,instructions))        
    dfjson=json.dumps([o.dump() for o in mylist])
    return dfjson

Cookdff=CookMan(df,user_ingredients)

0.6120870113372803


In [161]:
parsed =  json.loads(Cookdff)

In [164]:
parsed

[{'RecipeList': {'title': 'Lacha Onion Recipe - Masala Pickled Onions',
   'ingredients': 'Black Salt (Kala Namak) - as required,Salt - to taste,1/2 teaspoon Chaat Masala Powder,2 Onions - cut into thin rings,1 Lemon - juice extracted,1/2 teaspoon Red Chilli powder',
   'time': 10,
   'missing': 'chaat masala powder,lemon juice',
   'imgurl': 'https://www.archanaskitchen.com/images/archanaskitchen/0-Archanas-Kitchen-Recipes/2019/Masala_and_Plain_Lacha_Pyaz_Recipe_1.png',
   'cuisine': 'North Indian Recipes',
   'instructions': 'To begin making the Simple Lacha Pyaz Recipe - Pickled Onions  into a mixing bowl add the onion rings, lemon juice and sprinkle black salt to taste. Mix well to combine. Cover the bowl and allow the onions to rest in the lemon juice for about an hour.The Simple Lacha Pyaz Recipe - Pickled Onions are ready to be served after an hour.To make the Masala Lacha Pyaz Recipe - Pickled OnionsInto a mixing bowl add the onion rings, lemon juice, red chilli powder, chaat m

In [146]:
class Recipe: 
   
    # init method or constructor  
    def __init__(self, title,ing,time,missing,imgurl,cuisine,instructions): 
        self.title = title 
        self.ingredients=ing
        self.time=time
        self.missing=missing
        self.imgurl=imgurl
        self.cusine=cuisine
        self.instructions=instructions
    def dump(self):
        return {"RecipeList": {'title': self.title,
                               'ingredients': self.ingredients,
                               'time': self.time,
                               'missing': self.missing,
                               'imgurl':self.imgurl,
                               'cuisine':self.cuisine,
                               'instructions':self.instructions
                               }}

In [147]:
mylist=[]
for i in range(2):
     title = Cookdf.loc[i,"TranslatedRecipeName"]
     ing=Cookdf.loc[i,"TranslatedIngredients"]
     time=int(Cookdf.loc[i,"TotalTimeInMins"])
     missing=Cookdf.loc[i,"Missing"]
     imgurl =Cookdf.loc[i,"image-url"]
     cuisine=Cookdf.loc[i,"Cuisine"]
     instructions=Cookdf.loc[i,"TranslatedInstructions"]
     mylist.append(Recipe(title,ing,time,missing,imgurl,cuisine,instructions))

AttributeError: 'str' object has no attribute 'loc'

In [145]:
df.columns

Index(['TranslatedRecipeName', 'TranslatedIngredients', 'TotalTimeInMins',
       'Cuisine', 'TranslatedInstructions', 'URL', 'P-Ingredients', 'flag',
       'image-url', 'Ingredient-count'],
      dtype='object')

In [140]:
t=json.dumps([o.dump() for o in mylist])

In [142]:
t[1:-1]

'{"RecipeList": {"title": "Lacha Onion Recipe - Masala Pickled Onions", "ingredients": "Black Salt (Kala Namak) - as required,Salt - to taste,1/2 teaspoon Chaat Masala Powder,2 Onions - cut into thin rings,1 Lemon - juice extracted,1/2 teaspoon Red Chilli powder", "time": 10, "missing": "chaat masala powder,lemon juice", "imgurl": "https://www.archanaskitchen.com/images/archanaskitchen/0-Archanas-Kitchen-Recipes/2019/Masala_and_Plain_Lacha_Pyaz_Recipe_1.png"}}, {"RecipeList": {"title": "Porridge Bread Recipe", "ingredients": "Salt - to taste,1 cup Khichdi/Pongal,1/4 cup Rice flour - or as needed,1/4 cup Onion - finely diced (optional),Sunflower Oil - as needed", "time": 15, "missing": "khichdipongal,sunflower oil", "imgurl": "https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Gauravi_Vinay/Kichdi_rotti.jpg"}}'

## Code

In [165]:
import azure.functions as func
import pandas as pd
import numpy as np
import fuzzywuzzy.fuzz as fuzz
import os
import json


def CookMan(df,user_ingredients):
    
    class Recipe: 
   
 
        def __init__(self, title,ing,time,missing,imgurl,cuisine,instructions): 
            self.title = title 
            self.ingredients=ing
            self.time=time
            self.missing=missing
            self.imgurl=imgurl 
            self.cuisine=cuisine
            self.instructions=instructions            
        def dump(self):
            return {"RecipeList": {'title': self.title,
                               'ingredients': self.ingredients,
                               'time': self.time,
                               'missing': self.missing,
                               'imgurl':self.imgurl,
                               'cuisine':self.cuisine,
                               'instructions':self.instructions }}


    def Cook2(Pingredients):    
        "Comparing the Recipe Ingredients and User Ingredients and decreasing count value for each Recipe ingredient missing.\
        Count describes the no. of ingredients missing from user ingredients to make that recipe."
        flag,count=0,0
        ulcount=0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if i==j.strip():               
                    flag=1
                    ulcount=1
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
            if count<-10:
                return count
        if(ulcount==0):
            return count-1
        else:
            return count

    def MainCook(Pingredients):    
        flag,count=0,0
        ullcount=0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j.strip())>80:                 
                    flag=1 
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
        if(ullcount==0):
            return count-1
        else:
            return count        
    
    
    df["flag"]=df["P-Ingredients"].apply(Cook2)  
    df=df.sort_values(by="flag",ascending=False).head(100)
    df["flag2"]=df["P-Ingredients"].apply(MainCook)
    df=df.sort_values(by="flag2",ascending=False)

    def Missing(Pingredients):
        flag=0
        miss=""
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j.strip())>80:                    
                    flag=1 
                    break
            if (flag==0):
                miss+=i+","
                
            else:
                flag=0
        miss = miss[:-1]
        return miss
    df["Missing"]=df["P-Ingredients"].apply(Missing)
    df.drop(columns=["flag2","flag","P-Ingredients","URL"],inplace=True)
    df.reset_index(inplace=True,drop=True)
    Cookdf=df.head(30)
    mylist=[]
    for i in range(5):
        title = Cookdf.loc[i,"TranslatedRecipeName"]
        ing=Cookdf.loc[i,"TranslatedIngredients"]
        time=int(Cookdf.loc[i,"TotalTimeInMins"])
        missing=Cookdf.loc[i,"Missing"]
        imgurl = Cookdf.loc[i,"image-url"]
        cuisine=Cookdf.loc[i,"Cuisine"]
        instructions=Cookdf.loc[i,"TranslatedInstructions"]
        mylist.append(Recipe(title,ing,time,missing,imgurl,cuisine,instructions))        
    dfjson=json.dumps([o.dump() for o in mylist])
    return dfjson

Cookdff=CookMan(df,user_ingredients)


def main(req: func.HttpRequest):
    
    trial = req.params.get('trial')
    if trial == "yes":
        return func.HttpResponse("function runs")
    # else:
    #     return func.HttpResponse("function still runs bitch")
    df=pd.read_csv(os.path.join("df-en-final.csv"))
    df["Ingredient-count"]=df["P-Ingredients"].apply(lambda x:len(x.split(",")))
    df=df[df["Ingredient-count"]>4]
    df.reset_index(drop=True, inplace=True)
    req_body = req.get_json()
    user_ingredients=req_body.get('foodItems')
    result=CookMan(df,user_ingredients)
    parsed = json.loads(result)
    return func.HttpResponse(json.dumps(parsed, indent=2))

ModuleNotFoundError: No module named 'azure'

In [166]:
df=pd.read_csv("df-en-final.csv")

In [167]:
C

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,P-Ingredients,flag,image-url
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",-5,https://www.archanaskitchen.com/images/archana...
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",-9,https://www.archanaskitchen.com/images/archana...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",-10,https://www.archanaskitchen.com/images/archana...
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",-11,https://www.archanaskitchen.com/images/archana...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,sprig curry,as...",-8,https://www.archanaskitchen.com/images/archana...
...,...,...,...,...,...,...,...,...,...
5946,Saffron Paneer Peda Recipe,"2 teaspoons Rose water,1/2 teaspoon Ghee,2 cup...",20,Indian,To begin making the Saffron Paneer Peda recipe...,https://www.archanaskitchen.com/saffron-paneer...,"saffron strands generous,ghee,paneer,rose wate...",-7,https://www.archanaskitchen.com/images/archana...
5947,Italian Arancini Rice Balls Recipe With Delici...,"4 sprig Basil leaves - finely chopped,2 cups W...",100,Italian Recipes,To begin making the Italian Arancini Rice Ball...,https://www.archanaskitchen.com/italian-aranci...,"salt,gorgonzola cheese cubes,wheat bread crumb...",-11,https://www.archanaskitchen.com/images/archana...
5948,Quinoa Phirnee Recipe (Quinoa Milk Pudding),"3/4 cup Sugar,2 cup Milk - vegans can substitu...",35,Indian,"To begin making Quinoa Phirnee Recipe, place a...",https://www.archanaskitchen.com/quinoa-phirnee...,"ghee,milk,saffron strands,sugar,cardamom powde...",-8,https://www.archanaskitchen.com/images/archana...
5949,Ullikadala Pulusu Recipe | Spring Onion Curry,"1 pinch Turmeric powder (Haldi),1/2 teaspoon M...",15,Andhra,To begin making Ullikadala Pulusu Recipe | Spr...,https://www.archanaskitchen.com/ullikadala-pul...,"salt,tamarind water,mustard seeds,asafoetida (...",-9,https://www.archanaskitchen.com/images/archana...
